In [1]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
from time import time
import numpy as np
import pandas as pd
import string
from numpy.random import RandomState
import urllib.parse
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\risha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\risha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df=pd.read_csv('train.txt')

In [3]:
df

,tweet_id,sentiment,tweet_text
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going t...
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome c...
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...
...,...,...,...
21460,522949024132112384,neutral,"the day after newark ill be able to say """"i me..."
21461,522372593312350209,neutral,FEC hold farewell session for seven ministers ...
21462,522515200592052224,neutral,Luca Di Montezemolo (who's last day was Monday...
21463,523089087155437568,positive,Coffee is pretty much the answer to all questi...


In [4]:
np.unique(df['sentiment'].values,return_counts=True)

(array(['negative', 'neutral', 'positive'], dtype=object),
 array([3387, 9014, 9064], dtype=int64))

In [5]:
text_list=df['tweet_text'].values.tolist()

In [6]:
for i in range(20):
    print('------------------------------------------------------------------------------')
    print(text_list[i])

------------------------------------------------------------------------------
Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :)
------------------------------------------------------------------------------
Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.
------------------------------------------------------------------------------
its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.
------------------------------------------------------------------------------
Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out)
------------------------------------------------------------------------------
Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney
------------------------------------------------------------------------------
I 

In [7]:
test=pd.read_csv('test_samples.txt')

In [8]:
test

,tweet_id,tweet_text
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor..."
1,218775148495515649,Musical awareness: Great Big Beautiful Tomorro...
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour ana...
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susa..."
4,171874368908050432,"Excuse the connectivity of this live stream, f..."
...,...,...
5393,210378118865756160,It's a Wednesday girls night out as '90's band...
5394,245177521304399872,"night college course sorted, just have to enro..."
5395,259280987089932288,For the 1st time in 30 years. For your splendi...
5396,201113950211940352,NURSES DAY - 12 MAY 2012. Nursing: The heart b...


In [9]:
def url_parser(string):
  new_string = ''
  for i in string.split():
     s, n, p, pa, q, f = urllib.parse.urlparse(i)
     if s and n:
         pass
     elif i[:1] == '@':
         pass
     elif i[:1] == '#':
         new_string = new_string.strip() + ' ' + i[1:]
     else:
         new_string = new_string.strip() + ' ' + i
  return new_string

In [10]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [11]:
corpus=[]
from tqdm import tqdm
for i in tqdm(range(0,df.shape[0])):
    text=df['tweet_text'][i]
    text=url_parser(text)
    #print(text)
    text=re.sub('[^a-zA-Z]',' ',text)
    #print(text)
    text=re.sub(r"http\S+", "", text)
    #print(text)
    text=text.lower()
    #print(text)
    text=emoji_pattern.sub(r'', text)
    #print(text)
    text=text.split()
    #print(text)
    lemmatizer = WordNetLemmatizer()
    text=[lemmatizer.lemmatize(word,pos ="a") for word in text if not word in set(stopwords.words('english'))]
    #print(text)
    #ps=PorterStemmer()
    #text=[ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    #print(text)
    text=' '.join(text)
    #print(text)
    corpus.append(text)

100%|███████████████████████████████████████████████████████████████████████████| 21465/21465 [02:00<00:00, 178.73it/s]


In [12]:
corpus_test=[]
from tqdm import tqdm
for i in tqdm(range(0,test.shape[0])):
    text=test['tweet_text'][i]
    text=url_parser(text)
    #print(text)
    text=re.sub('[^a-zA-Z]',' ',text)
    #print(text)
    text=re.sub(r"http\S+", "", text)
    #print(text)
    text=text.lower()
    #print(text)
    text=emoji_pattern.sub(r'', text)
    #print(text)
    text=text.split()
    #print(text)
    lemmatizer = WordNetLemmatizer()
    text=[lemmatizer.lemmatize(word,pos ="a") for word in text if not word in set(stopwords.words('english'))]
    #print(text)
    #ps=PorterStemmer()
    #text=[ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    #print(text)
    text=' '.join(text)
    #print(text)
    corpus_test.append(text)

100%|█████████████████████████████████████████████████████████████████████████████| 5398/5398 [00:28<00:00, 188.22it/s]


In [14]:
# del corpus
# del corpus_test

In [16]:
df['corpus']=corpus
test['corpus']=corpus_test

In [17]:
df

,tweet_id,sentiment,tweet_text,corpus
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going t...,gas house hit u going chapel hill sat
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa an...,theo walcott still shit u c watch rafa johnny ...
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just h...,u gsp fan u c hate nick diaz u wait february
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome c...,iranian general says israel u iron dome u deal...
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...,tehran u c mon amour obama tried establish tie...
...,...,...,...,...
21460,522949024132112384,neutral,"the day after newark ill be able to say """"i me...",day newark ill able say met demi lovato yester...
21461,522372593312350209,neutral,FEC hold farewell session for seven ministers ...,fec hold farewell session seven ministers pres...
21462,522515200592052224,neutral,Luca Di Montezemolo (who's last day was Monday...,luca di montezemolo last day monday alonso lea...
21463,523089087155437568,positive,Coffee is pretty much the answer to all questi...,coffee pretty much answer questions today frid...


In [18]:
test

,tweet_id,tweet_text,corpus
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor...",atlantic city ventnor margate ocean city area ...
1,218775148495515649,Musical awareness: Great Big Beautiful Tomorro...,musical awareness great big beautiful tomorrow...
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour ana...,radio fm fri oct labour analyst shawn hattingh...
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susa...",kapan sih lo ngebuktiin jan ngomong doang susa...
4,171874368908050432,"Excuse the connectivity of this live stream, f...",excuse connectivity live stream baba amr many ...
...,...,...,...
5393,210378118865756160,It's a Wednesday girls night out as '90's band...,wednesday girls night band wilson phillips fil...
5394,245177521304399872,"night college course sorted, just have to enro...",night college course sorted enrole tomorrow fi...
5395,259280987089932288,For the 1st time in 30 years. For your splendi...,st time years splendiferous entertainment arts...
5396,201113950211940352,NURSES DAY - 12 MAY 2012. Nursing: The heart b...,nurses day may nursing heart beat health


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
#from sklearn.model_selection import train_test_split

In [ ]:
# x_train,x_test,y_train,y_test=train_test_split(df['corpus'].values,df['sentiment'].values,test_size=0.2)

In [17]:
cv=CountVectorizer()
train_cv=cv.fit_transform(df['corpus'].values)


In [18]:
train_cv.shape

(21465, 26874)

In [22]:
tfidf=TfidfVectorizer()
train_tfidf=tfidf.fit_transform(df['corpus'].values)
test_tfidf=tfidf.transform(test['corpus'].values)

In [23]:
train_tfidf.shape

(21465, 26874)

In [24]:
from time import time

# Support Vector Machines

In [26]:
from sklearn.svm import SVC
svm=SVC(kernel='rbf')
from time import time
start=time()
svm.fit(train_tfidf,df['sentiment'].values)
end=time()

In [27]:
print(f'Time taken to train is {end-start} seconds ')

Time taken to train is 138.57800006866455 seconds 


In [28]:
svm_predictions=svm.predict(test_tfidf)

# Naive Bayes Classifier

In [29]:
from sklearn.naive_bayes import ComplementNB

In [30]:
cnb=ComplementNB()
from time import time
start=time()
cnb.fit(train_tfidf,df['sentiment'].values)
end=time()
print(f'Time taken to train is {end-start} seconds ')

Time taken to train is 0.054999351501464844 seconds 


In [33]:
cnb_predictions=cnb.predict(test_tfidf)

In [41]:
from sklearn.naive_bayes import MultinomialNB

In [42]:
mnb=MultinomialNB()
from time import time
start=time()
mnb.fit(train_tfidf,df['sentiment'].values)
end=time()
print(f'Time taken to train is {end-start} seconds ')

Time taken to train is 0.04703474044799805 seconds 


In [47]:
mnb_predictions=mnb.predict(test_tfidf)

In [38]:
svm_pred_df=pd.DataFrame({'tweet_id':test['tweet_id'].values,'sentiment':svm_predictions})
svm_pred_df.to_csv('svm_preds.csv',index=False)

In [46]:
cnb_pred_df=pd.DataFrame({'tweet_id':test['tweet_id'].values,'sentiment':cnb_predictions})
cnb_pred_df.to_csv('cnb_preds.csv',index=False)

In [48]:
mnb_pred_df=pd.DataFrame({'tweet_id':test['tweet_id'].values,'sentiment':mnb_predictions})
mnb_pred_df.to_csv('mnb_preds.csv',index=False)